## Task 04. Color Object Detection 

Now we have a more complex scenario. We have several objects of the same colour, i.e. 5 yellow balls. They are diposed around the scene having diferent sizes. What we want now is to track the only one that is moving around, no matter its size. 


### Goals:
* Step 1: Basic motion detection 

* Step 2: Detect the presence of colored objects using computer vision techniques.

* Step 3: Track the object as it moves around in the video frames, drawing its previous positions as it moves, creating a tail behind it

# Initialization parameters

In [1]:
# import the necessary packages
from collections import deque
import numpy as np
import time
import cv2
import datetime

#Convenience resize function
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation=inter)

    # return the resized image
    return resized

# Define the threshold of the color

In [3]:
VIDEODEV = 0
camera = cv2.VideoCapture(VIDEODEV); assert camera.isOpened()
time.sleep(0.25)
firstFrame = None


def setup_trackbars(range_filter):
    cv2.namedWindow("Trackbars", 0)
    for i in ["MIN", "MAX"]:
        for j in range_filter:
            v = 0 if i == "MIN" else 255
            cv2.createTrackbar("%s_%s" % (j, i), "Trackbars", v, 255, lambda x : None)
            
            

def get_trackbar_values(range_filter):
    values = {}

    for i in ["MIN", "MAX"]:
        for j in range_filter:
            v = cv2.getTrackbarPos("%s_%s" % (j, i), "Trackbars")
            values["%s_%s" % (j, i)] = v
    return values

camera = cv2.VideoCapture(VIDEODEV); assert camera.isOpened()
setup_trackbars('HSV')
while True:
        (grabbed, frame) = camera.read()
        frame_to_thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        v = get_trackbar_values('HSV')
        thresh = cv2.inRange(frame_to_thresh, (v['H_MIN'], v['S_MIN'], v['V_MIN']), (v['H_MAX'], v['S_MAX'], v['V_MAX']))
        cv2.imshow("Original", frame)
        cv2.imshow("Thresh", thresh)
        if cv2.waitKey(1) & 0xFF is ord('q'):
            break
            
print "Lower = (%d,%d,%d)" % (v['H_MIN'], v['S_MIN'], v['V_MIN'])
print "Upper = (%d,%d,%d)" % (v['H_MAX'], v['S_MAX'], v['V_MAX'])
# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()

Lower = (0,0,0)
Upper = (255,255,255)


# Track the movement of the colored item

To fulfill this commitment, we mixed the two codes (trackball & basicmotion) and changed the definition of background. Instead of having a "real" background, what we will do will be refresh it each $n$ frames in order to know what items are really moving at that moment. So if there are 2 balls on the frame it will detect one or none, depending if they are moving or not.


In [4]:

##
colorLower = (115,109,217)
colorUpper = (227,181,255)
bg_rate = 50 # Number of frames after the background is refreshed
##


VIDEODEV = 0
camera = cv2.VideoCapture(VIDEODEV); assert camera.isOpened()
time.sleep(0.25)
firstFrame = None

tailsize = 64
pts = deque(maxlen=tailsize)
it = 0

min_area = 500
diffDelta = 25
camera = cv2.VideoCapture(VIDEODEV); assert camera.isOpened()
time.sleep(0.25)
while True:
    (grabbed, frame) = camera.read()
    text = "Unoccupied"
    
    
    frame = resize(frame, width=500)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    if firstFrame == None: firstFrame = gray
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, diffDelta, 255, cv2.THRESH_BINARY)[1]
    
    mask = cv2.inRange(hsv, colorLower, colorUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    thresh = cv2.dilate(thresh, None, iterations=2)
    
    # The difference betwen now and what we did earlier, is that in this case we are detecting
    # the contours on the image times the mask, so all the regions of the background are
    # "invisible" in this step
    cnts = cv2.findContours(thresh.copy()*mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None

    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        try:
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            if radius > 10:
                cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
                cv2.circle(frame, center, 5, (0, 0, 255), -1)
                cv2.putText(frame, "x: {}, y: {}".format(center[0], center[1]),(10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.35, (0, 0, 255), 1)
        except:
            print "Divided by 0"
        
    pts.appendleft(center)
    
    if it%bg_rate ==0: # Refresh the backgound step
        firstFrame = gray

    for i in xrange(1, len(pts)):
        if pts[i - 1] is None or pts[i] is None:
            continue
        thickness = int(np.sqrt(tailsize / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
    cv2.imshow("Security Feed", frame)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", firstFrame)
    it +=1
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key is pressed, break from the lop
    if key == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()

Divided by 0
